# Продвинутые примеры: Нетривиальные системы

В этом туториале мы исследуем **сложные системы переписывания** с интересными математическими свойствами.

## 🎯 Что вы изучите

1. **Арифметическая система** — правила для унарной арифметики
2. **Циклическая система** — правила с нетривиальными ω-пределами
3. **Хаотическая система** — правила с высокой энтропией
4. **Сравнительный анализ** — метрики разных систем

## 🔬 Цель

Понять, как структура правил влияет на:
- Размер достижимого множества
- Топологию графа переходов
- Геометрические характеристики
- Вероятностные распределения

In [14]:
import sys
from pathlib import Path
import math

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

from rewriting import String, Rule, RewritingEngine
from integration import SpaceLanguageOrchestrator, PipelineConfig

def print_full_analysis(name: str, result, rules: list):
    """Выводит полный анализ системы в удобочитаемом виде."""
    
    print()
    print("╔" + "═"*70 + "╗")
    print(f"║  ПОЛНЫЙ АНАЛИЗ: {name:^50}  ║")
    print("╚" + "═"*70 + "╝")
    print()
    
    # L0: СИСТЕМА ПЕРЕПИСЫВАНИЯ
    print("┌" + "─"*70 + "┐")
    print("│  L0: СИСТЕМА ПЕРЕПИСЫВАНИЯ" + " "*42 + "│")
    print("└" + "─"*70 + "┘")
    print()
    print(f"  Правил: {len(rules)}")
    for i, rule in enumerate(rules, 1):
        print(f"    R{i}: {rule.left} → {rule.right}")
    print()
    print(f"  Достижимых строк: {len(result.reachable_strings)}")
    print(f"  Нормальных форм: {len(result.normal_forms)}")
    print(f"  ω-предел: {len(result.omega_limit)} строк")
    print()
    
    # Показываем достижимые строки
    print("  Достижимые строки:")
    sorted_strings = sorted(result.reachable_strings, key=lambda x: (len(str(x)), str(x)))
    for i, s in enumerate(sorted_strings[:15]):
        marker = " ◄ нормальная форма" if s in result.normal_forms else ""
        print(f"    '{s}'{marker}")
    if len(sorted_strings) > 15:
        print(f"    ... ещё {len(sorted_strings) - 15} строк")
    print()
    
    # L2: ГРАФ И SCC
    print("┌" + "─"*70 + "┐")
    print("│  L2: ГРАФ ПЕРЕХОДОВ И SCC" + " "*43 + "│")
    print("└" + "─"*70 + "┘")
    print()
    print(f"  Вершин (строк):     {result.graph_vertices}")
    print(f"  Рёбер (переходов):  {result.graph_edges}")
    print(f"  SCC компонент:      {result.scc_count}")
    print(f"  Наибольшая SCC:     {result.largest_scc_size} вершин")
    print()
    if result.largest_scc_size == 1:
        print("  → Интерпретация: нет циклов, система терминирует")
    else:
        print(f"  → Интерпретация: есть циклы (SCC размера {result.largest_scc_size})")
    print()
    
    # L4: МАКРОСЫ
    print("┌" + "─"*70 + "┐")
    print("│  L4: МАКРОСЫ (повторяющиеся паттерны)" + " "*31 + "│")
    print("└" + "─"*70 + "┘")
    print()
    if result.macros and len(result.macros) > 0:
        print(f"  Найдено макросов: {len(result.macros)}")
        for i, macro in enumerate(list(result.macros)[:10], 1):
            print(f"    {i}. '{macro}'")
    else:
        print("  Макросов не найдено")
    print()
    
    # L7: МАТЕМАТИЧЕСКИЕ ПОДЪЁМЫ
    print("┌" + "─"*70 + "┐")
    print("│  L7: МАТЕМАТИЧЕСКИЕ ПОДЪЁМЫ" + " "*41 + "│")
    print("└" + "─"*70 + "┘")
    print()
    
    # Геометрия
    print("  📐 ГЕОМЕТРИЯ:")
    if result.geometry_data:
        dim = result.geometry_data.get('dimension', 'N/A')
        curv = result.geometry_data.get('curvature', 0)
        pts = result.geometry_data.get('points', 0)
        print(f"      Размерность: {dim}, Кривизна: {curv:.4f}, Точек: {pts}")
    else:
        print("      (данные недоступны)")
    
    # Меры
    print("  📊 МЕРЫ:")
    if result.measure_data:
        entropy = result.measure_data.get('entropy', 0)
        support = result.measure_data.get('support_size', 0)
        print(f"      Энтропия: {entropy:.3f} бит, Носитель: {support} строк")
    else:
        print("      (данные недоступны)")
    
    # Категории
    print("  🎯 КАТЕГОРИИ:")
    if result.category_data:
        objs = result.category_data.get('objects', 0)
        morphs = result.category_data.get('morphisms', 0)
        print(f"      Объектов: {objs}, Морфизмов: {morphs}")
    else:
        print("      (данные недоступны)")
    print()
    
    if result.errors:
        print("  ⚠️ Предупреждения:", ", ".join(result.errors[:3]))

print("✓ Модули загружены")
print("✓ Функция print_full_analysis() готова")

✓ Модули загружены
✓ Функция print_full_analysis() готова


---

## Пример 1️⃣: Арифметическая система

Система правил для унарной арифметики. Символы:
- `0` — единица
- `|` — маркер/разделитель

Правила моделируют группировку и упрощение:

In [15]:
# Арифметические правила
arithmetic_rules = [
    Rule(String.from_str("00"), String.from_str("|")),   # R1: 00 → | (сжатие)
    Rule(String.from_str("||"), String.from_str("0")),   # R2: || → 0 (свёртка)
]

print("Арифметическая система:")
for i, rule in enumerate(arithmetic_rules, 1):
    print(f"  R{i}: {rule.left} → {rule.right}")

# Начальная строка
initial_arith = String.from_str("0000")
print(f"\nНачальная строка: '{initial_arith}'")

Арифметическая система:
  R1: 00 → |
  R2: || → 0

Начальная строка: '0000'


### Полный анализ арифметической системы

In [16]:
# Конфигурация для арифметической системы
config_arith = PipelineConfig(
    graph_depth=10,
    max_graph_size=200,
    min_macro_frequency=2,
    min_macro_length=2,
    enable_z3=False,
    enable_isabelle=False,
    enable_geometry=True,
    enable_measure=True,
    enable_category=True,
    embedding_dim=8
)

# Запуск анализа
orch_arith = SpaceLanguageOrchestrator(arithmetic_rules, config_arith)
result_arith = orch_arith.analyze(initial_arith, verbose=False)

# Полный вывод результатов
print_full_analysis("АРИФМЕТИЧЕСКАЯ СИСТЕМА", result_arith, arithmetic_rules)


╔══════════════════════════════════════════════════════════════════════╗
║  ПОЛНЫЙ АНАЛИЗ:               АРИФМЕТИЧЕСКАЯ СИСТЕМА                ║
╚══════════════════════════════════════════════════════════════════════╝

┌──────────────────────────────────────────────────────────────────────┐
│  L0: СИСТЕМА ПЕРЕПИСЫВАНИЯ                                          │
└──────────────────────────────────────────────────────────────────────┘

  Правил: 2
    R1: 00 → |
    R2: || → 0

  Достижимых строк: 6
  Нормальных форм: 2
  ω-предел: 1 строк

  Достижимые строки:
    '0' ◄ нормальная форма
    '||'
    '00|'
    '0|0' ◄ нормальная форма
    '|00'
    '0000'

┌──────────────────────────────────────────────────────────────────────┐
│  L2: ГРАФ ПЕРЕХОДОВ И SCC                                           │
└──────────────────────────────────────────────────────────────────────┘

  Вершин (строк):     6
  Рёбер (переходов):  6
  SCC компонент:      6
  Наибольшая SCC:     1 вершин

  → Интерпрет

---

## Пример 2️⃣: Циклическая система

Система с **замкнутыми циклами** — правила образуют петли переходов.

Особенность: из строки можно вернуться в неё же через несколько шагов!

In [17]:
# Циклические правила
cyclic_rules = [
    Rule(String.from_str("0"), String.from_str("|")),    # R1: 0 → |
    Rule(String.from_str("|"), String.from_str("0|")),   # R2: | → 0|
    Rule(String.from_str("0|"), String.from_str("||")),  # R3: 0| → ||
    Rule(String.from_str("||"), String.from_str("0")),   # R4: || → 0
]

print("Циклическая система:")
for i, rule in enumerate(cyclic_rules, 1):
    print(f"  R{i}: {rule.left} → {rule.right}")
    
print()
print("Цикл: 0 → | → 0| → || → 0 → ...")

# Начальная строка
initial_cyclic = String.from_str("0")
print(f"\nНачальная строка: '{initial_cyclic}'")

Циклическая система:
  R1: 0 → |
  R2: | → 0|
  R3: 0| → ||
  R4: || → 0

Цикл: 0 → | → 0| → || → 0 → ...

Начальная строка: '0'


### Полный анализ циклической системы

In [18]:
# Конфигурация для циклической системы (ограничиваем глубину из-за роста)
config_cyclic = PipelineConfig(
    graph_depth=5,       # Меньше глубина — система быстро растёт
    max_graph_size=100,
    min_macro_frequency=2,
    min_macro_length=2,
    enable_z3=False,
    enable_isabelle=False,
    enable_geometry=True,
    enable_measure=True,
    enable_category=True,
    embedding_dim=8
)

# Запуск анализа
orch_cyclic = SpaceLanguageOrchestrator(cyclic_rules, config_cyclic)
result_cyclic = orch_cyclic.analyze(initial_cyclic, verbose=False)

# Полный вывод результатов
print_full_analysis("ЦИКЛИЧЕСКАЯ СИСТЕМА", result_cyclic, cyclic_rules)


╔══════════════════════════════════════════════════════════════════════╗
║  ПОЛНЫЙ АНАЛИЗ:                ЦИКЛИЧЕСКАЯ СИСТЕМА                  ║
╚══════════════════════════════════════════════════════════════════════╝

┌──────────────────────────────────────────────────────────────────────┐
│  L0: СИСТЕМА ПЕРЕПИСЫВАНИЯ                                          │
└──────────────────────────────────────────────────────────────────────┘

  Правил: 4
    R1: 0 → |
    R2: | → 0|
    R3: 0| → ||
    R4: || → 0

  Достижимых строк: 176
  Нормальных форм: 0
  ω-предел: 100 строк

  Достижимые строки:
    '0'
    '|'
    '00'
    '0|'
    '|0'
    '||'
    '000'
    '00|'
    '0|0'
    '0||'
    '|00'
    '|0|'
    '||0'
    '|||'
    '0000'
    ... ещё 161 строк

┌──────────────────────────────────────────────────────────────────────┐
│  L2: ГРАФ ПЕРЕХОДОВ И SCC                                           │
└──────────────────────────────────────────────────────────────────────┘

  Вершин (стро

---

## Пример 3️⃣: Хаотическая система

Система с **высоким недетерминизмом** — из одной строки можно перейти в несколько разных!

In [ ]:
# Хаотические правила (много недетерминизма)
chaotic_rules = [
    Rule(String.from_str("0"), String.from_str("|")),    # R1: 0 → |
    Rule(String.from_str("0"), String.from_str("||")),   # R2: 0 → || (альтернатива!)
    Rule(String.from_str("|"), String.from_str("0")),    # R3: | → 0
    Rule(String.from_str("|"), String.from_str("00")),   # R4: | → 00 (альтернатива!)
    Rule(String.from_str("00"), String.from_str("|")),   # R5: 00 → |
    Rule(String.from_str("||"), String.from_str("0")),   # R6: || → 0
]

print("Хаотическая система:")
for i, rule in enumerate(chaotic_rules, 1):
    print(f"  R{i}: {rule.left} → {rule.right}")

print()
print("Недетерминизм: 0 может стать | или ||, а | может стать 0 или 00")

initial_chaotic = String.from_str("0|")
print(f"\nНачальная строка: '{initial_chaotic}'")

Хаотическая система:
  R1: 0 → |
  R2: 0 → ||
  R3: | → 0
  R4: | → 00
  R5: 00 → |
  R6: || → 0

✓ Начальная строка: 0|


In [20]:
# Анализ хаотической системы
config_chaotic = PipelineConfig(
    graph_depth=5,
    max_graph_size=150,
    enable_measure=True,
    embedding_dim=12
)

orch_chaotic = SpaceLanguageOrchestrator(chaotic_rules, config_chaotic)
result_chaotic = orch_chaotic.analyze(initial_chaotic, verbose=False)

print(f"✓ Хаотическая система проанализирована")
print(f"  Достижимо: {len(result_chaotic.reachable_strings)} строк")
print(f"  Граф: {result_chaotic.graph_vertices} вершин, {result_chaotic.graph_edges} рёбер")
if result_chaotic.measure_data:
    entropy = result_chaotic.measure_data.get('entropy', 0)
    print(f"  Энтропия: {entropy:.3f} бит (высокая хаотичность!)")

✓ Хаотическая система проанализирована
  Достижимо: 5458 строк
  Граф: 5458 вершин, 92834 рёбер
  Энтропия: 12.414 бит (высокая хаотичность!)


## 📊 Сравнительный анализ

Сведём метрики всех трёх систем в единую таблицу и сравним их характеристики по различным параметрам:

In [21]:
# Собираем данные для сравнения
systems_data = {
    "Система": ["Арифметическая", "Циклическая", "Хаотическая"],
    "Правил": [len(arithmetic_rules), len(cyclic_rules), len(chaotic_rules)],
    "Достижимо": [
        len(result_arith.reachable_strings),
        len(result_cyclic.reachable_strings),
        len(result_chaotic.reachable_strings)
    ],
    "Вершин": [
        result_arith.graph_vertices,
        result_cyclic.graph_vertices,
        result_chaotic.graph_vertices
    ],
    "Рёбер": [
        result_arith.graph_edges,
        result_cyclic.graph_edges,
        result_chaotic.graph_edges
    ],
    "SCC": [
        result_arith.scc_count,
        result_cyclic.scc_count,
        result_chaotic.scc_count
    ],
    "Энтропия": [
        result_arith.measure_data.get('entropy', 0) if result_arith.measure_data else 0,
        result_cyclic.measure_data.get('entropy', 0) if result_cyclic.measure_data else 0,
        result_chaotic.measure_data.get('entropy', 0) if result_chaotic.measure_data else 0
    ]
}

# Форматированный вывод таблицы
print("СРАВНИТЕЛЬНАЯ ТАБЛИЦА")
print("="*80)
print(f"{'Система':<20} {'Правил':>8} {'Достижимо':>12} {'Вершин':>10} {'Рёбер':>10} {'SCC':>6} {'Энтропия':>10}")
print("-"*80)
for i in range(len(systems_data["Система"])):
    print(f"{systems_data['Система'][i]:<20} "
          f"{systems_data['Правил'][i]:>8} "
          f"{systems_data['Достижимо'][i]:>12} "
          f"{systems_data['Вершин'][i]:>10} "
          f"{systems_data['Рёбер'][i]:>10} "
          f"{systems_data['SCC'][i]:>6} "
          f"{systems_data['Энтропия'][i]:>10.3f}")
print("="*80)

СРАВНИТЕЛЬНАЯ ТАБЛИЦА
Система                Правил    Достижимо     Вершин      Рёбер    SCC   Энтропия
--------------------------------------------------------------------------------
Арифметическая              2            6          6          6      6      2.585
Циклическая                 4          176        176        856     30      7.459
Хаотическая                 6         5458       5458      92834      0     12.414


## 🔬 Геометрические характеристики

Сравним геометрические лифты систем — размерности многообразий, кривизну и количество точек в геометрическом представлении:

In [22]:
print("ГЕОМЕТРИЧЕСКИЕ ХАРАКТЕРИСТИКИ")
print("="*80)

for name, result in [
    ("Арифметическая", result_arith),
    ("Циклическая", result_cyclic),
    ("Хаотическая", result_chaotic)
]:
    print(f"\n{name} система:")
    
    if result.geometry_data:
        dim = result.geometry_data.get('dimension', 'N/A')
        curv = result.geometry_data.get('curvature', 0)
        pts = result.geometry_data.get('points', 0)
        print(f"  📐 Размерность: {dim}")
        print(f"  📐 Кривизна: {curv:.4f}")
        print(f"  📐 Точек: {pts}")
    else:
        print("  (геометрия не вычислена)")
        
print("="*80)

ГЕОМЕТРИЧЕСКИЕ ХАРАКТЕРИСТИКИ

Арифметическая система:
  📐 Размерность: 1
  📐 Кривизна: 0.0402
  📐 Точек: 6

Циклическая система:
  📐 Размерность: 2
  📐 Кривизна: 0.0720
  📐 Точек: 20

Хаотическая система:
  📐 Размерность: 2
  📐 Кривизна: 0.0512
  📐 Точек: 20


## 💡 Ключевые наблюдения

На основе сравнительного анализа выделим закономерности и важные выводы о поведении систем переписывания:

In [23]:
print("КЛЮЧЕВЫЕ НАБЛЮДЕНИЯ")
print("="*80)

# Энтропии
entropies = [
    ("Арифметическая", result_arith.measure_data.get('entropy', 0) if result_arith.measure_data else 0),
    ("Циклическая", result_cyclic.measure_data.get('entropy', 0) if result_cyclic.measure_data else 0),
    ("Хаотическая", result_chaotic.measure_data.get('entropy', 0) if result_chaotic.measure_data else 0)
]

max_entropy_sys = max(entropies, key=lambda x: x[1])
min_entropy_sys = min(entropies, key=lambda x: x[1])

print(f"\n1. Самая упорядоченная: {min_entropy_sys[0]} (энтропия {min_entropy_sys[1]:.3f} бит)")
print(f"   → Низкая энтропия = высокая предсказуемость")

print(f"\n2. Самая хаотичная: {max_entropy_sys[0]} (энтропия {max_entropy_sys[1]:.3f} бит)")
print(f"   → Высокая энтропия = много вариантов развития")

# SCC анализ
print(f"\n3. Циклическая структура:")
print(f"   - Циклическая система: {result_cyclic.largest_scc_size} вершин в наибольшей SCC")
print(f"   - Указывает на замкнутые аттракторы")

# Размер графа
print(f"\n4. Сложность графа:")
print(f"   - Хаотическая: {result_chaotic.graph_edges} рёбер (самая связная)")
print(f"   - Арифметическая: {result_arith.graph_edges} рёбер")
print(f"   - Больше рёбер = больше недетерминизма")

print("="*80)

КЛЮЧЕВЫЕ НАБЛЮДЕНИЯ

1. Самая упорядоченная: Арифметическая (энтропия 2.585 бит)
   → Низкая энтропия = высокая предсказуемость

2. Самая хаотичная: Хаотическая (энтропия 12.414 бит)
   → Высокая энтропия = много вариантов развития

3. Циклическая структура:
   - Циклическая система: 147 вершин в наибольшей SCC
   - Указывает на замкнутые аттракторы

4. Сложность графа:
   - Хаотическая: 92834 рёбер (самая связная)
   - Арифметическая: 6 рёбер
   - Больше рёбер = больше недетерминизма


---

## 📊 Резюме

В этом туториале мы исследовали три качественно разных системы:

| Система | Характер | Ключевая особенность |
|---------|----------|----------------------|
| **Арифметическая** | Детерминированная | Всегда терминирует в нормальной форме |
| **Циклическая** | Периодическая | Образует замкнутые циклы (SCC > 1) |
| **Хаотическая** | Недетерминированная | Много альтернативных путей |

### Что показывают метрики:

- **Энтропия** — чем выше, тем больше "свободы" в системе
- **SCC** — размер показывает наличие и величину циклов
- **Кривизна** — геометрическая неоднородность пространства строк
- **Рёбра/вершины** — степень недетерминизма

### Практическое значение:

1. **Терминация** — арифметическая система гарантированно заканчивается
2. **Циклы** — циклическая система может работать вечно
3. **Непредсказуемость** — хаотическая система требует перебора всех путей

## 🚀 Эксперименты

Попробуйте создать собственные системы и сравнить их характеристики!